In [ ]:
# Some useful Google API documentation:
# https://developers.google.com/maps/documentation/directions/
# https://developers.google.com/maps/documentation/roads/snap
from modules.API_Keys import StreetView_API_Key
import os
from urllib.request import urlretrieve, urlopen
import json
import pandas as pd
import cv2
import math
import time
import sys
import requests
import math
apikey_streetview = StreetView_API_Key

In [ ]:
def get_sv_video(apikey_streetview, start_coords, end_coords, savepath='PHOTO_FOLDER/', video_name='street_view_video.avi', fps=3,section=3):
    distance = haversine(start_coords, end_coords)
    num_images = max(1, int(distance // 30))  # At least 1 image for every 3 feet

    lat_step = (end_coords[0] - start_coords[0]) / num_images
    lon_step = (end_coords[1] - start_coords[1]) / num_images

    images = []
    metadata = [] 
    
    pic_dim = "640x640"  # Adjusted for efficiency
    if section in [5,6]:
        direction=90
    else:
        direction= 0
    for i in range(num_images):
        lat = start_coords[0] + lat_step * i
        lon = start_coords[1] + lon_step * i
        # time.sleep(5)
        # Generate filename with current time in nanoseconds
        timestamp = time.time_ns()
        filename = f"image_{timestamp}"
        
        get_sv_img(apikey_streetview, (lat, lon), filename=filename, savepath=savepath, picsize=pic_dim, heading=direction)
        
        image_path = os.path.join(savepath, filename + ".jpg")
        metadata.append({
            "image_name": filename + ".jpg",
            "latitude": lat,
            "longitude": lon,
            "image_size": pic_dim
        })
        
        images.append(image_path)

    # Creating video from images
    if images:
        frame = cv2.imread(images[0])
        height, width, layers = frame.shape
        video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'XVID'), fps, (width, height))

        for image in images:
            video.write(cv2.imread(image))

        cv2.destroyAllWindows()
        video.release()

    # Save metadata
    metadata_df = pd.DataFrame(metadata)
    metadata_df.to_csv(os.path.join(savepath, 'metadata.csv'), index=False)


In [ ]:
def process_road_sections(apikey_streetview, road_sections, savepath='PHOTO_FOLDER/'):
    for idx, (start_coords, end_coords) in enumerate(road_sections):
        print(f"Processing section {idx + 1}: Start {start_coords}, End {end_coords}")
        video_name = os.path.join(savepath, f'street_view_video_section_{idx + 1}.avi')
        get_sv_video(apikey_streetview, start_coords, end_coords, savepath=savepath, video_name=video_name, section=idx + 1)


In [ ]:

# Example list of road sections (tuples of (start_coords, end_coords))
road_sections = [
    ((25.085294619594887, 55.19778624258285), (25.09331469279902, 55.203831529713725)),
    ((25.09331469279902, 55.203831529713725), (25.09485548254961, 55.20371683004338)),
    ((25.095551165722455, 55.20329182526299), (25.09843033276794, 55.20316293376894)),
    ((25.09843033276794, 55.20316293376894), (25.099856922008204, 55.20430863594598)),
    ((25.09983098417138, 55.20430863594598), (25.10014223785042, 55.20592694026025)),
    ((25.10014223785042, 55.20592694026025), (25.100505429726603, 55.21165847724775)),
    ((25.101339654372964, 55.212643485672885), (25.110188596136346, 55.219896698011766)),
    ((25.110188596136346, 55.219896698011766), (25.11375096639641, 55.220001823943896)),
    ((25.115373091455645, 55.22080099313514), (25.1476527907927, 55.24685799361344)),
    ]


process_road_sections(apikey_streetview, road_sections, savepath='PHOTO_FOLDER/')
